In [1]:
import numpy as np
import torch
import os
import cv2
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import PIL
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from skimage import feature
from imutils import build_montages
from imutils import paths
from numpy import loadtxt
from xgboost import XGBClassifier

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip '/content/drive/MyDrive/parkinsons.zip'

Archive:  /content/drive/MyDrive/parkinsons.zip
  inflating: drawings/spiral/testing/healthy/V01HE01.png  
  inflating: drawings/spiral/testing/healthy/V02HE01.png  
  inflating: drawings/spiral/testing/healthy/V03HE1.png  
  inflating: drawings/spiral/testing/healthy/V04HE01.png  
  inflating: drawings/spiral/testing/healthy/V05HE01.png  
  inflating: drawings/spiral/testing/healthy/V06HE01.png  
  inflating: drawings/spiral/testing/healthy/V07HE01.png  
  inflating: drawings/spiral/testing/healthy/V08HE01.png  
  inflating: drawings/spiral/testing/healthy/V09HE01.png  
  inflating: drawings/spiral/testing/healthy/V10HE01.png  
  inflating: drawings/spiral/testing/healthy/V11HE01.png  
  inflating: drawings/spiral/testing/healthy/V55HE12.png  
  inflating: drawings/spiral/testing/healthy/V55HE13.png  
  inflating: drawings/spiral/testing/healthy/V55HE14.png  
  inflating: drawings/spiral/testing/healthy/V55HE15.png  
  inflating: drawings/spiral/testing/parkinson/V01PE01.png  
  infla

In [4]:
def HOG(image):
	features = feature.hog(image, orientations=9,pixels_per_cell=(10, 10), cells_per_block=(2, 2),
		                      transform_sqrt=True, block_norm="L1")
	return features

In [5]:
def Dataloader(path):
  imgpaths = list(paths.list_images(path))
  imgs,labels = [],[]
  for i in imgpaths:
    img = cv2.imread(i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (128,128))
    img = cv2.threshold(img, 0,255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    imfeat = HOG(img)

    imgs.append(imfeat)
    labels.append(i.split('/')[4])
  return np.array(imgs), np.array(labels)

In [6]:
xtrain,ytrain = Dataloader('/content/wave/training')
xtest,ytest = Dataloader('/content/wave/testing')
sxtrain,sytrain = Dataloader('/content/spiral/training')
sxtest, sytest = Dataloader('/content/spiral/testing')

In [7]:
print(len(xtrain))

72


In [8]:
print(len(ytrain))

72


In [9]:
le = LabelEncoder()
ytrain = le.fit_transform(ytrain)
ytest = le.fit_transform(ytest)
le = LabelEncoder()
sytrain = le.fit_transform(sytrain)
sytest = le.fit_transform(sytest)

In [11]:
model = XGBClassifier()
model.fit(xtrain, ytrain)

XGBClassifier()

In [12]:
y_pred = model.predict(xtest)
predictions = [round(value) for value in y_pred]

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
# evaluate predictions
accuracy = accuracy_score(ytest, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 56.67%


In [16]:
cm = confusion_matrix(ytest,y_pred)
print(cm)

[[ 6  9]
 [ 4 11]]


In [17]:
(tn,fp),(fn,tp) = cm
precision = tp / (fp + tp)
recall = tp / (fn + tp)
f1_score = 2*precision*recall / (precision + recall)


In [18]:
print('Wave classification using Logistic Regression \nPrecision: '+ str(round(precision,4)) + '\nRecall: ' + str(round(recall,4)) + '\nF1 Score: ' + str(round(f1_score,4)))


Wave classification using Logistic Regression 
Precision: 0.55
Recall: 0.7333
F1 Score: 0.6286


In [19]:
accuracy = (tn+tp) / float(cm.sum())
print('Wave classification using Logistic Regression \nAccuracy value:', accuracy)

Wave classification using Logistic Regression 
Accuracy value: 0.5666666666666667
